data collected April 25, 2020 on a DigitalOcean droplet running Ubuntu 18.04

In [1]:
!pip install python-geoip-python3

In [14]:
import pandas
print('pandas',pandas.__version__)
import re
import time
from geoip import geolite2
import sys
print(sys.version)
import matplotlib.pyplot as plt

pandas 0.23.4
3.6.6 | packaged by conda-forge | (default, Oct 12 2018, 14:08:43) 
[GCC 4.8.2 20140120 (Red Hat 4.8.2-15)]


In [3]:
with open('nginx_access.log','r') as f:
    lines = f.readlines()

In [4]:
lines[0:5]

['71.244.214.232 - - [25/Apr/2020:21:25:50 +0000] "GET / HTTP/1.1" 200 17635 "-" "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.113 Safari/537.36" "-"\n',
 '71.244.214.232 - - [25/Apr/2020:21:25:50 +0000] "GET /static/sorttable.js HTTP/1.1" 200 16877 "http://167.172.250.37/" "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.113 Safari/537.36" "-"\n',
 '71.244.214.232 - - [25/Apr/2020:21:25:51 +0000] "GET /static/d3.min.js HTTP/1.1" 200 247829 "http://167.172.250.37/" "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.113 Safari/537.36" "-"\n',
 '71.244.214.232 - - [25/Apr/2020:21:25:51 +0000] "GET /static/single_step.png HTTP/1.1" 200 14494 "http://167.172.250.37/" "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.113 Safari/537.36" "-"\n',
 '71.244.214.232 - - [25/Ap

In [32]:
list_of_dicts = []
for indx, line in enumerate(lines):
    line_as_dict = {}
    line_as_list = line.split(' ')
    line_as_dict['remote ip'] = line_as_list[0]
    line_as_dict['remote user'] = line_as_list[2]
    line_as_dict['date time'] = line_as_list[3]
    message = ' '.join(line_as_list[5:])
    # https://stackoverflow.com/questions/16710076/python-split-a-string-respect-and-preserve-quotes
    msg_as_list = re.findall(r'(?:[^\s,"]|"(?:\\.|[^"])*")+', message)
    if len(msg_as_list)!=6:
        print(message)
    request = msg_as_list[0]
    request_as_list = request.split(' ')
    if (len(request_as_list)!=3):
        print('request does not match expected format:')
        print(line)
    else:
        line_as_dict['request method'] = request_as_list[0].replace('"','')
        line_as_dict['request file'] = request_as_list[1]
        line_as_dict['request proto'] = request_as_list[2].replace('"','')
    line_as_dict['status'] = msg_as_list[1]
    line_as_dict['body bytes sent'] = msg_as_list[2]
    line_as_dict['referrer'] = msg_as_list[3]
    line_as_dict['user agent'] = msg_as_list[4]
    line_as_dict['gzip ratio'] = msg_as_list[5]

    list_of_dicts.append(line_as_dict)

request does not match expected format:
42.231.98.143 - - [26/Apr/2020:01:26:00 +0000] "sh+/tmp/gpon80&ipv=0" 400 157 "-" "-" "-"

request does not match expected format:
66.240.205.34 - - [26/Apr/2020:10:10:32 +0000] "H\x00\x00\x00tj\xA8\x9E#D\x98+\xCA\xF0\xA7\xBBl\xC5\x19\xD7\x8D\xB6\x18\xEDJ\x1En\xC1\xF9xu[l\xF0E\x1D-j\xEC\xD4xL\xC9r\xC9\x15\x10u\xE0%\x86Rtg\x05fv\x86]%\xCC\x80\x0C\xE8\xCF\xAE\x00\xB5\xC0f\xC8\x8DD\xC5\x09\xF4" 400 157 "-" "-" "-"

request does not match expected format:
223.149.7.214 - - [26/Apr/2020:10:15:19 +0000] "/tmp/gpon80&ipv=0" 400 157 "-" "-" "-"



In [33]:
df = pandas.DataFrame(list_of_dicts)
df.shape

(835, 11)

In [34]:
# https://strftime.org/
df['datetime'] = pandas.to_datetime(df['date time'], format='[%d/%b/%Y:%H:%M:%S')
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop.html
df.drop('date time', inplace=True, axis=1)

In [35]:
df.head()

,body bytes sent,gzip ratio,referrer,remote ip,remote user,request file,request method,request proto,status,user agent,datetime
0,17635,"""-""","""-""",71.244.214.232,-,/,GET,HTTP/1.1,200,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_...",2020-04-25 21:25:50
1,16877,"""-""","""http://167.172.250.37/""",71.244.214.232,-,/static/sorttable.js,GET,HTTP/1.1,200,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_...",2020-04-25 21:25:50
2,247829,"""-""","""http://167.172.250.37/""",71.244.214.232,-,/static/d3.min.js,GET,HTTP/1.1,200,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_...",2020-04-25 21:25:51
3,14494,"""-""","""http://167.172.250.37/""",71.244.214.232,-,/static/single_step.png,GET,HTTP/1.1,200,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_...",2020-04-25 21:25:51
4,1619,"""-""","""http://167.172.250.37/""",71.244.214.232,-,/static/884319.json,GET,HTTP/1.1,200,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_...",2020-04-25 21:25:51


In [43]:
df['remote ip'].value_counts()

35.197.133.35      312
96.245.195.226     237
162.129.252.228    143
71.244.214.232      51
36.90.214.103       36
150.136.227.32       9
129.213.139.225      9
35.229.249.135       6
106.52.178.254       2
42.231.98.143        2
223.149.7.214        2
66.240.205.34        1
185.221.3.200        1
185.49.97.183        1
118.173.190.27       1
113.162.186.4        1
177.154.58.141       1
191.242.246.205      1
85.51.57.203         1
202.178.115.76       1
148.63.179.102       1
1.1.162.136          1
88.227.248.203       1
190.94.140.165       1
47.101.221.252       1
218.149.209.48       1
162.243.130.100      1
93.139.158.31        1
138.118.100.199      1
177.105.235.38       1
70.76.233.21         1
115.238.44.237       1
166.114.1.8          1
84.51.123.41         1
45.13.93.82          1
183.134.104.170      1
193.118.53.210       1
Name: remote ip, dtype: int64

In [42]:
df['request proto'].value_counts()

HTTP/1.1    830
HTTP/1.0      2
Name: request proto, dtype: int64

In [41]:
df['user agent'].value_counts()

"Mozilla/5.0 (X11; Linux x86_64; rv:68.0) Gecko/20100101 Firefox/68.0"                                                                           237
"Mozilla/5.0 (iPhone; CPU iPhone OS 13_3_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.0.5 Mobile/15E148 Safari/604.1"    143
"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36"                            114
"Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.0)"                                                                                              92
"Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:47.0) Gecko/20100101 Firefox/47.0"                                                                    78
"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.113 Safari/537.36"                       51
"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.88 Safa

In [40]:
df['status'].value_counts()

404    369
200    304
304    149
400      9
302      2
500      2
Name: status, dtype: int64

In [38]:
df['request method'].value_counts()

GET        703
POST       128
CONNECT      1
Name: request method, dtype: int64

In [37]:
df['request file'].value_counts()

/static/mathjax/tex-svg.js                                         40
/static/sorttable.js                                               40
/                                                                  39
/static/d3.min.js                                                  13
/static/declareinitialexpr.png                                     12
/static/dividebothsidesby.png                                      12
/static/declarefinalexpr.png                                       12
/editor                                                            12
/static/multiplybothsidesby.png                                    11
/static/3131111133.png                                              8
/static/single_step.png                                             8
/static/9040079362.png                                              8
/static/9565166889.png                                              8
/static/884319.json                                                 8
/static/2113211456.p